# Перед запуском!
Для работы matplotlib+ipywidgets необходима библиотека ipympl!

In [1]:
#pip install ipympl

# Задание №2. Image Enhancement 1
1. Загрузите цветное изображение, преобразуйте BGR к RGB и отобразите
2. Пребразуйте его к градациям серого
3. Постройте гистограмму
4. Попробуйте улучшить изображение (в т.ч., ухудшить, и потом улучшить): 
	- contrast stretching в заданном диапазоне,
	- equalizeHist,
	- гамма-коррекция, ...

In [2]:
import cv2
%matplotlib widget
from ipywidgets import HBox, VBox, Output, SelectionRangeSlider as srs, ToggleButtons as tb, IntSlider, FloatSlider
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import urllib

req = urllib.request.urlopen('https://upload.wikimedia.org/wikipedia/ru/thumb/2/24/Lenna.png/800px-Lenna.png')
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
lena = cv2.imdecode(arr, -1)

In [3]:
class Lab():
    def __init__(self, img):
        self.img_original=img
        self.img = self.img_original
        
        self.output=Output(layout = {'height': '80%',
                                     'border': '1px solid black'},)
        with self.output:
            self.fig, self.ax = plt.subplots(figsize=(5, 5), ncols=1, nrows=2, constrained_layout=True)

        self.bgrrgb_tb=tb(
            options=['BGR', 'RGB'],
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltips=['BGR', 'RGB'],
            continuous_update=False
        )
    
        self.contrast_sldr=srs(
            options=list(range(0,513)),
            index=(0,256),
            description='Контраст',
            disabled=False,
            continuous_update=False
        )
        
        self.size_sldr=IntSlider(
            min=0,
            max=1024,
            step=10,
            description='Размер:',
            continuous_update=False
        )
        
        self.blur_sldr=IntSlider(
            min=1,
            max=15,
            step=2,
            description='MedianBlur:',
            continuous_update=False
        )
        
        self.gamma_sldr=FloatSlider(
            min=0.25,
            max=5,
            step=0.25,
            description='Гамма:',
            continuous_update=False
        )

        self.reset()
        
        self.bgrrgb_tb.observe(self.update)
        self.contrast_sldr.observe(self.update)
        self.size_sldr.observe(self.update)
        self.blur_sldr.observe(self.update)
        self.gamma_sldr.observe(self.update)
        
        self.menu=VBox([self.bgrrgb_tb, self.contrast_sldr, self.size_sldr, self.blur_sldr, self.gamma_sldr],
                      layout = {'width': '*'})
        self.out=HBox([self.menu,self.output])

        self.update(None)
    
    def __repr__(self):
        display(self.out)
        return ''
    
    def reset(self):
        self.img=self.img_original
        self.contrast_sldr.index=(0, 256)
        self.size_sldr.value=self.img.shape[0]
        self.blur_sldr.value=1
        self.gamma_sldr.value=1
    
    def change_format(self):
        if self.bgrrgb_tb.value=='BGR':
            return self.img_original
        return cv2.cvtColor(self.img_original, cv2.COLOR_BGR2RGB)
        
    def change_range(self):
        rng=self.contrast_sldr.value
        range_width = 1 if rng[1]==rng[0] else rng[1] - rng[0] #Заглушка от деления на нуль
        hist_range = 256 / range_width, -rng[0]
        return cv2.convertScaleAbs(self.img, None, hist_range[0], hist_range[1])

    
    def change_size(self):
        size=self.size_sldr.value
        return cv2.resize(self.img, (size,size), interpolation=cv2.INTER_LINEAR)
    
    def change_blur(self):
        blur=self.blur_sldr.value
        return cv2.medianBlur(self.img, blur)

    def change_gamma(self):
        gamma=self.gamma_sldr.value
        invGamma = 1.0 / gamma if gamma!=0 else 1
        lut=np.array([((i/255.0) ** invGamma) * 255
                    for i in np.arange(0,256)]).astype('uint8')
        return cv2.LUT(self.img, lut)
    
    def hist(self, img):
        return cv2.calcHist([img], [0],None,[256],[0,256])
    
    def update(self, value):
        self.img=self.change_format()
        self.img=self.change_range()
        self.img=self.change_size()
        self.img=self.change_blur()
        self.img=self.change_gamma()
        self.ax[0].clear()
        self.ax[0].imshow(self.img)
        self.ax[1].clear()
        self.ax[1].plot(self.hist(self.img), color='black')

In [4]:
a=Lab(lena)
a